In [71]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

#import the data
local_path = '/Users/scoot/Documents/GitHub/CU-Sabermetrics-Final/SQL/rawData2010_2013_v2.csv'
df = pd.read_csv(local_path)
df.head()

,GAME_DT,GAME_ID,battingTeam,fieldingTeam,player,pitchSequence,balls,strikes,EVENT_CD,inning,...,lN2,outsOnPlay,runsOnPlay,ballparkID,homeTeam,homeHits,homeScore,awayTeam,awayHits,awayScore
0,100405.0,ANA201004050,ANA,MIN,aybae001,BBCCFBFFFB,3,2,14,1,...,1B only,0,0,ANA01,ANA,9.0,6.0,MIN,7.0,3.0
1,100405.0,ANA201004050,ANA,MIN,abreb001,CX,0,1,2,1,...,1B only,1,0,ANA01,ANA,9.0,6.0,MIN,7.0,3.0
2,100405.0,ANA201004050,ANA,MIN,huntt001,B,0,0,9,1,...,2B only,0,0,ANA01,ANA,9.0,6.0,MIN,7.0,3.0
3,100405.0,ANA201004050,ANA,MIN,huntt001,B.B2BB,3,0,14,1,...,1B & 2B,0,0,ANA01,ANA,9.0,6.0,MIN,7.0,3.0
4,100405.0,ANA201004050,ANA,MIN,matsh001,CFX,0,2,2,1,...,1B & 2B,1,0,ANA01,ANA,9.0,6.0,MIN,7.0,3.0


In [72]:
#initializing the useful columns:
batterlistUnique = pd.DataFrame(df.player.unique())
batterlistUnique.columns = ['player']
batterlistUnique['outs'] = 0
batterlistUnique['runs'] = 0
# batterlistUnique['endOutState'] = 99
# batterlistUnique['strBaseState'] = 100
# batterlistUnique['endBaseState'] = 100
#print(batterlistUnique)

In [73]:
#testing setting certain value, weird indexing stuff
# batterlistUnique.loc[batterlistUnique['player']=='spand001', 'outs'] = 101
# print(batterlistUnique)

In [74]:
#print(batterlistUnique)
counter = 0
for i in range(len(batterlistUnique)):
    if batterlistUnique['outs'][i] > 0:
    #if batterlistUnique.loc[batterlistUnique[item],'outs'] > 0:
        #print(batterlistUnique.loc[batterlistUnique[item],'outs'])
        counter += 1
print(counter)
        

0


In [75]:
# -- CHECKING IF ALGO RUNS CORRECTLY
dfM = df.head(1000)
blU = pd.DataFrame(dfM.player.unique())
blU.columns = ['player']
blU['outScore'] = 0
blU['runScore'] = 0
blU['baseScore'] = 0
# -- checking if this "filters" by player
temp = dfM[dfM['player']=='aybae001']
temp = temp.reset_index(drop=True)
print(sum(temp['outsOnPlay']) )
# print(temp)

# -- MARGINAL OUT WEIGHT: need to mess with based on dist of outs
mOut = dfM['outsOnPlay'].replace(0, np.NaN).mean()
mOutWeight = 1/3


# -- MARGINAL RUN WEIGHT: need to mess with based on dist of runs
mRun = dfM['runsOnPlay'].replace(0, np.NaN).mean()
mRunW = (1)

for batter in blU['player']:
    temp = dfM[dfM['player']==batter]
    temp = temp.reset_index(drop=True)
    for i in range(len(temp['GAME_ID'])):
        # -- OUTS:
        if(temp['outsOnPlay'][i] > 0):
            blU.loc[blU['player'] == batter,'outScore'] += (temp['outsOnPlay'][i]/mOut) # * (temp['inning'][i]) * mOutW)
            # -- SAC RUN:
            if(temp['runsOnPlay'][i] > 0):
                 blU.loc[blU['player'] == batter,'runScore'] += (temp['runsOnPlay'][i]/mRun)/(temp['outsOnPlay'][i]) # * (temp['inning'][i]) * mRunW)
            # -- SAC BASES:
            
        
        # -- NO OUTS: 
        elif(temp['outsOnPlay'][i] == 0):
            # -- RUNS:
            if(temp['runsOnPlay'][i] > 0):
                blU.loc[blU['player'] == batter,'runScore'] += (temp['runsOnPlay'][i]/mRun)  # /temp['runsOnPlay'][i]) * (temp['inning'][i]) * mRunW)
            
            # -- BASES:
            
            
print(blU)


36
      player   outScore  runScore  baseScore
0   aybae001  34.623529  0.807018          0
1   abreb001  36.547059  2.421053          0
2   huntt001  39.432353  4.035088          0
3   matsh001  33.661765  6.859649          0
4   kendh001  29.814706  3.228070          0
5   morak001  32.700000  8.070175          0
6   rivej001  33.661765  4.035088          0
7   woodb003  27.891176  0.000000          0
8   mathj001  13.464706  2.421053          0
9   spand001  15.388235  0.807018          0
10  hudso001  12.502941  0.000000          0
11  mauej001   9.617647  2.421053          0
12  mornj001   7.694118  2.421053          0
13  cuddm001  10.579412  0.000000          0
14  kubej002   7.694118  0.807018          0
15  yound003  10.579412  4.842105          0
16  hardj003  13.464706  1.614035          0
17  puntn001   6.732353  0.807018          0
18  thomj002   6.732353  2.421053          0
19  harrb001   2.885294  1.614035          0
20  iztum001  16.350000  3.228070          0
21  nap